# Plotly Tools

可互動式畫圖套件 

In [9]:
import pandas as pd
import statistics
import datetime 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb
import codecs
import re
import os
import pickle
from datetime import datetime


path    = './Fund_Data/'
dic     = {}

# 找尋所有資料夾內的基金資料
for dirPath, dirNames, fileNames in os.walk(path):
    for f in fileNames:
        if bool(re.search(r"_all.csv$",f)):
            name = re.findall(r"\w+(?:_)",f)[0][0:-1]
            dic[name] = os.path.join(dirPath, f)
            print('基金: '+ name)

# 讀全部檔案
dfs = [pd.read_csv(dic[file], skiprows=3, names=['年月日', file + " 淨值(元)"]) for file in dic]

# 把全部基金的 dataframe 合併成一個
df_merge = dfs[0]
for df_ in dfs[1:]:
    df_merge = df_merge.merge(df_, on='年月日', how='left')

基金: 野村美利高收債A
基金: 野村美利高收債AS
基金: 野村美利高收債BC
基金: 野村美利高收債B1
基金: 野村美利高收債B2


In [6]:
df_merge.head()

,年月日,野村美利高收債A 淨值(元),野村美利高收債AS 淨值(元),野村美利高收債BC 淨值(元),野村美利高收債B1 淨值(元),野村美利高收債B2 淨值(元)
0,9/11/2015,9.9999,NaN,9.9967,9.9999,10.0007
1,10/11/2015,9.9996,NaN,9.9919,9.9996,9.9956
2,11/11/2015,10.0010,NaN,9.9972,10.0010,9.9981
3,12/11/2015,9.9938,NaN,9.9819,9.9938,9.9809
4,13/11/2015,9.9880,NaN,9.9654,9.9880,9.9734


## Installl plotly

In [2]:
#!pip install plotly

# 基金資料畫圖

In [23]:
from plotly.graph_objs import Scatter, Layout
import plotly.graph_objs as go
import plotly 

# 把時間資料轉為 rangeselector 可讀取的格式 Ex: 2019-05-03
x_data = []
for x in df_merge['年月日']:
    x_data.append(datetime.strptime(x, "%d/%m/%Y").strftime("%Y-%m-%d"))

# 只在 Jupyter 裡面畫圖，不公開
plotly.offline.init_notebook_mode(connected=True)

# 初始化圖
fig = go.Figure()

# 把每檔基金的圖畫出
for col in df_merge.columns[1:]:
    fig.add_trace(go.Scatter(x=x_data, y=df_merge[col], mode='lines', name=col[:-5]))
    
# 標題名稱與位置
fig.update_layout(title_text='基金淨值', title_x=0.5)


# 時間區段按鈕
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1M", step="month", stepmode="backward"),
            dict(count=6, label="6M", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1Y", step="year", stepmode="backward"),
            dict(count=2, label="2Y", step="year", stepmode="backward"),
            dict(label="All",step="all")
        ])
    )
)

plotly.offline.iplot(fig)